# BA780 Analyzing Inpatient medical expense in US

## Contents

### 1. Introduction
##### 1.1 Data source
##### 1.2 Objectives
##### 1.3 Term Explanation

### 2. Data Preparation
##### 2.1 Data Pre-processing
##### 2.2 Data Overview

### 3. Exploratory Data Analysis
##### 3.1 Total Payment by State
##### 3.2 Medicare Payment by State
##### 3.3 Coverage Rate by State
##### 3.4 Coverage Rate Findings
##### 3.5 Life Expectancy/Income
##### 3.6 Prediction of Average total payment

### 4. Conclusions

### 5. References

## 1. Introduction

#### 1.1 Data source

We collect our data from CMS(Centers for Medicare & Medicaid Services) official website. CMS is a federal agency within the United States Department of Health and Human Services (HHS) that administers the Medicare program and works in partnership with state governments to administer Medicaid, the Children's Health Insurance Program (CHIP), and health insurance portability standards. We are going to focus on analyzing the data 'Medicare Provider Utilization and Payment Data: Inpatient' from the year 2011 to 2017. Moreover, we collect the United state's household income by states and life expectancy by state data from the CDC(Centers for Disease Control and Prevention) and the United States Census Bureau. We will use the US household income from 2010 to 2017 by states and the life expectancy data including estimates of U.S. life expectancy at birth by state and census tract for the period 2010-2015. 

#### 1.2 Objectives

Our goal is to analyze the inpatient expense of US medicare. We would like to explore the relations of covered rate, medicare charges, and total payments with regions, using visualization for medical charges and payments in different states from 2011 to 2017. Based on the exploratory analysis results, we will further utilize a machine learning model to predict total payments using relevant features. In addition, we are going to find and to analyze the relationship between inpatient expense and household income, inpatient expense, and life expectancy. 

#### 1.3 Term Explanation

The following terms are 5 primary factors that we use to analyze the inpatient expense of US medicare. 
- Total payments: all expenses that the hospital charged to patients(amount of patient paid + amount of medicare paid) 
- Medicare payment: the amount of expenses that paid by Medicare
- Cover rate: Medicare payment/Total Payments
- Life expectancy: is a measure of the average time an individual is expected to live
- Income: we have use the median of household income for our following analytics

## 2. Data Preparation

#### 2.1 Data Pre-processing

##### Import packages

In [ ]:
pip install plotly

In [ ]:
pip install kaleido

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go

In [ ]:
inpatient_2015=pd.read_csv('inpatient_2015.csv')
inpatient_2017=pd.read_csv('Inpatient_2017.csv')
life_expectancy = pd.read_csv('US_life_expectancy.csv')
payment_7years = pd.read_csv('patments_by_state_in_7_years.csv')
inpatient_payment_by_year = pd.read_csv('inpatient_payment_by_year.csv')
income = pd.read_csv('ACSST1Y2015.S1901_data_with_overlays_2020-11-19T221147.csv')

In [ ]:
inpatient_2015.info()

In [ ]:
inpatient_2015.head()

In [ ]:
life_expectancy.info()

In [ ]:
life_expectancy.head()

In [ ]:
income = income[['Geographic Area Name', 'Households!!Estimate!!Median income (dollars)']]
income.rename(columns={'Geographic Area Name': "state", 'Households!!Estimate!!Median income (dollars)': "household_median_income"}, inplace=True)
fullname = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", 
            "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", 
            "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
            "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", 
            "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
            "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
shortname = ["AK", "AL", "AR", "AZ",  "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "IA", "ID", "IL", "IN",  "KS", "KY", "LA", "MA", "MD", "ME",  
           "MI", "MN", "MO", "MS", "MT","NC","ND", "NE","NH","NJ","NM", "NV",   
           "NY", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VA", "VT",  "WA", "WI", "WV", "WY"]
income.sort_values(by='state',ascending=True,inplace=True)
state = pd.DataFrame({'state':fullname, 'abbreviation':shortname})
income_state = state.merge(income,on='state')
income_state = income_state.drop('state',axis=1)

In [ ]:
income_state.info()

In [ ]:
income_state.head()

#### 2.2 Data Overview

At the beginning, we will look at the overall average level of medical payments in the United States from 2011 to 2017.

In [ ]:
pivot_table = payment_7years.pivot_table(index ='provider_state', values=['average_total_payments','average_medicare_payments','cover_rate'])
pivot_table.round(2).head()

In [ ]:
table1=payment_7years.sort_values('average_total_payments')
g1 = sns.catplot(kind='bar',x="provider_state",y='average_total_payments',data=table1,height=2.6, aspect=5)
g1.fig.suptitle("Avgerage Total Payments in 7 years by state")

In [ ]:
table2=payment_7years.sort_values('average_medicare_payments')
g2 = sns.catplot(kind='bar',x="provider_state",y='average_medicare_payments',data=table2 ,height=2.6, aspect=5)
g2.fig.suptitle("Avgerage Medicare Payments in 7 years by state")

The two charts above show the average medicare payment has consistency with the average total payment in these 7 years we researched, for example, the states with the smallest or the largest amount of medicare payment are those have the smallest or the largest amount of total payment.


In [ ]:
table3=payment_7years.sort_values('cover_rate')
g3 = sns.catplot(x="provider_state",y='cover_rate',data=table3 ,height=2.6, aspect=5)
g3.fig.suptitle("Average Cover Rate in 7 years by state")

As a whole, each state’s cover rate has little to do with medical payments and total payments, for example, as one of the most expensive medicare and total payment states, such as Hi and DC, become one of the states with the lowest cover rate, but AK and MD retain the high level of cover rate with the expensive medicare and total payment.

The table below shows the average total payment through out the US from 2011 to 2017. As we mention previously, we insert a 'year' column for each inpatient table. We union the inpatient payment tables altogether and list the average total payment for each year from 2011 to 2017. We are trying to see how does the US overall average total payments change throughout the times. As the list result shows, the average total payment increased each year from 2011 to 2017, especially that there was a significant increase during the year 2013 to 2014. The reason behind this was probably due to the Patient Protection and Affordable Care Act, and commonly known as Obamacare. The act came up in 2010 and implemented in 2014. 

In [ ]:
pm_by_year = inpatient_payment_by_year[['year','average_total_payments']]
pm_by_year = pm_by_year.groupby('year').mean()
pm_by_year

In [ ]:
sns.relplot(kind='line', marker='o', data=pm_by_year,height=3, aspect=5)
plt.title('Average Total Payment in USA by years')
plt.ylabel('dollar($)')
sns.despine()
#test#

In [ ]:
pm_by_year_state = inpatient_payment_by_year[['year','provider_state','average_total_payments']] 
sns.catplot(x="year", y="average_total_payments", data=pm_by_year_state, kind ='box',height=6, aspect=2)
plt.title('Average Total Payment in USA by years')
plt.ylabel('dollar($)')
sns.despine()

## 3. Exploratory Data Analysis

#### 3.1 What is the average total payment in each state in 2015?

In [ ]:
pm_2015 = inpatient_2015[['provider_state','average_total_payments']]
pm_2015 = pm_2015.groupby('provider_state').mean().sort_values(by='average_total_payments',ascending = False).reset_index()
pm_2015.head(5)

In [ ]:
pm_2015 = inpatient_2015[['provider_state','average_total_payments']]
pm_2015 = pm_2015.groupby('provider_state').mean().sort_values(by='average_total_payments',ascending = True).reset_index()
pm_2015.head(5)

As the table shows above, the TOP 5 average total payment states are Washington D.C(DC), Alaska(AK), Hawaii(HI), California(CA), and Maryland(MD). And the state with lowest average total cost is Alabama(AL) with $10001 in year 2015.

In [ ]:
fig = go.Figure(data = go.Choropleth(
    locations=pm_2015['provider_state'],
    z=pm_2015['average_total_payments'].astype(float),
    locationmode='USA-states',
    colorscale='BuPu',
    colorbar_title="Avg Total Payments",
    hovertext=pm_2015['provider_state'],
), layout=dict(title = 'Average Total Payment by States'))

fig.update_layout(
    geo_scope = 'usa'
)

fig.show('svg')

#### 3.2 How does Medicare perform in different states?

In [ ]:
medicare_payments = inpatient_2015[['provider_state','average_medicare_payments']]
medicare_payments_states = (medicare_payments.groupby('provider_state').mean()).sort_values(by="average_medicare_payments",ascending = False)
avg_nation_m = medicare_payments_states['average_medicare_payments'].mean()
medicare_payments_states['average_nationwide'] = avg_nation_m
medicare_payments_states = medicare_payments_states.eval('Relative_to_nationwide = average_medicare_payments/average_nationwide').round({'Relative_to_nationwide':2})
df_medicare = medicare_payments_states.drop(['average_nationwide'],axis=1).head()
df_medicare

In [ ]:
df_comb = df_medicare.merge(pm_2015,how='inner',on='provider_state')
med_top_state = df_comb.set_index('provider_state').sort_values(by='average_medicare_payments',ascending=False).head()
med_top_state.drop(['Relative_to_nationwide'],axis=1)

In [ ]:
total_top_state = df_comb.set_index('provider_state').sort_values(by='average_total_payments',ascending=False).head()
total_top_state.drop(['Relative_to_nationwide'],axis=1)

In [ ]:
df_plot = medicare_payments_states.drop(['Relative_to_nationwide'],axis=1)
sns.relplot(data=df_plot, kind = 'line',height=2.6, aspect=5)
plt.ylabel('Medicare Payments')
plt.title('Medicare payments in states compared to nationwide average in 2015')

After comparing the result in each state to the nationwide medicare payments, it is obvious that medicare payments adjust geographically. And about one third of states have a higher medicare payment than the average.

#### 3.3 What is the inpatient coverage rate in each state?

In [ ]:
cover_rate = inpatient_2015[['provider_state','average_medicare_payments','average_total_payments']]
c = cover_rate.groupby('provider_state').sum().eval('cover_rate = average_medicare_payments/ average_total_payments').round({'cover_rate':3})
c.reset_index()
df = c.sort_values('cover_rate', ascending=False).reset_index()
df = df[['provider_state','cover_rate']]
df.head(5)

In [ ]:
fig = go.Figure(data = go.Choropleth(
    locations=df['provider_state'],
    z=df['cover_rate'].astype(float),
    locationmode='USA-states',
    colorscale='BuPu',
    colorbar_title="Cover Rate",
    hovertext=df['provider_state'],
), layout=dict(title = 'Coverage rate by States'))

fig.update_layout(
    geo_scope = 'usa'
)

fig.show('svg')

We use medicare payments divided by total payments to get the coverage rate. We can see the coverage rate varies from state to state.

#### 3.4 Why are the coverage rates for each state different?

We noticed that the coverage rates for each state in 2015 ranges from 0.77 (DC) to 0.91 (MD), so we would like to find the reason for the differences. We seek to clinical conditions (DRG_Definition in data), and list the clinical conditions with the highest covered charges for states with top 5 and bottom 5 coverage rates.

Top clinical conditions for top 5 states by total covered charges:

In [ ]:
inpatient_top_states = inpatient_2015[inpatient_2015['provider_state'].isin(['MD', 'AK', 'CA', 'MA', 'MT'])]
inpatient_top_states_pivot = pd.pivot_table(inpatient_top_states, values='average_medicare_payments', index='drg_definition', columns='provider_state', aggfunc='sum')
inpatient_top_states_pivot = inpatient_top_states_pivot[['MD', 'AK', 'CA', 'MA', 'MT']]
inpatient_top_states_pivot = inpatient_top_states_pivot.sort_values('CA', ascending=False)
inpatient_top_states_pivot.head(10)

In [ ]:
# Filter top 30 clinical conditions
inpatient_top_states_pivot = inpatient_top_states_pivot.reset_index()
top_100_condition = inpatient_top_states_pivot.head(100)
inpatient_top_states_pivot = inpatient_top_states_pivot.head(30)

In [ ]:
# Use state as a single column for plotting purpose
inpatient_top_states_graph = pd.melt(inpatient_top_states_pivot, id_vars='drg_definition', value_vars=['MD', 'AK', 'CA', 'MA', 'MT'], value_name='total_covered_charges')

In [ ]:
# Plot top clinical conditions by states with top cover rates
size = (16, 10)
fig, ax = plt.subplots(figsize=size)
top_g = sns.scatterplot(data=inpatient_top_states_graph, x='total_covered_charges', y='drg_definition', hue='provider_state', alpha=0.7, ax=ax)
top_g.set(xlabel = 'Total Covered Charges', ylabel = 'Clinical Conditions')
plt.title('Top Clinical Conditions by States with Top Cover Rates')
plt.show(top_g)

Top clinical conditions for last 5 states by total covered charges:

In [ ]:
inpatient_last_states = inpatient_2015[inpatient_2015['provider_state'].isin(['DC', 'UT', 'PA', 'VA', 'OH'])]
inpatient_last_states_pivot = pd.pivot_table(inpatient_last_states, values='average_medicare_payments', index='drg_definition', columns='provider_state', aggfunc='sum')
inpatient_last_states_pivot = inpatient_last_states_pivot[['DC', 'UT', 'PA', 'VA', 'OH']]
inpatient_last_states_pivot = inpatient_last_states_pivot.sort_values('PA', ascending=False)
inpatient_last_states_pivot.head(10)

In [ ]:
# Filter top 30 clinical conditions, and use state as a single column for plotting purpose
inpatient_last_states_pivot = inpatient_last_states_pivot.reset_index()
top_100_condition = inpatient_top_states_pivot.head(100)
inpatient_last_states_pivot = inpatient_last_states_pivot.head(30)
inpatient_last_states_graph = pd.melt(inpatient_last_states_pivot, id_vars='drg_definition', value_vars=['DC', 'UT', 'PA', 'VA', 'OH'], value_name='total_covered_charges')

In [ ]:
# Plot top clinical conditions by states with lowest cover rates
size = (16, 10)
fig, ax = plt.subplots(figsize=size)
last_g = sns.scatterplot(data=inpatient_last_states_graph, x='total_covered_charges', y='drg_definition', hue='provider_state', alpha=0.7, ax=ax)
last_g.set(xlabel = 'Total Covered Charges', ylabel = 'Clinical Conditions')
plt.title('Top Clinical Conditions by States with Lowest Cover Rates')
plt.show(last_g)

The clinical conditions with the most covered charges for each state are similar, including major joint replacement, septicemia, heart failure, etc. Therefore, we further examine the coverage rates for the common top clinical conditions for those 10 states, and find out that coverage rates for those clinical conditions are good indicators of overall coverage rates for each state. 

In [ ]:
# Find cover rate of each clinical condition in each state
cover_rate_drg = inpatient_2015[['provider_state','drg_definition', 'average_medicare_payments','average_total_payments']]
c_clinical_conditions_state = cover_rate_drg.groupby(['drg_definition', 'provider_state']).sum().eval('cover_rate = average_medicare_payments/ average_total_payments').round({'cover_rate':3})
c_clinical_conditions_state.reset_index(inplace=True)
c_clinical_conditions_state.head()

In [ ]:
# Find the order of clinical conditions (drg_definition) by total covered charges
order = pd.pivot_table(cover_rate_drg, index='drg_definition', values='average_medicare_payments', aggfunc='sum').sort_values('average_medicare_payments', ascending=False).reset_index()
order_drg = list(order['drg_definition'])[:30]

In [ ]:
# Filter by 10 states and top 30 conditions
c_compare = c_clinical_conditions_state.loc[
                                            (c_clinical_conditions_state['provider_state'].isin(['MD', 'AK', 'CA', 'MA', 'MT', 'OH', 'VA', 'PA', 'UT', 'DC'])) &
                                            (c_clinical_conditions_state['drg_definition'].isin(order_drg))
                                            ]

# Transform the date for cover rate heatmap
c_compare_pivot = pd.pivot_table(c_compare, values='cover_rate', index='drg_definition', columns='provider_state')

# Reorder the states by descending cover rates, and clinical conditions by total covered charges
top_drg = pd.Index(order_drg)
c_compare_pivot = c_compare_pivot.loc[top_drg, ['MD', 'AK', 'CA', 'MA', 'MT', 'OH', 'VA', 'PA', 'UT', 'DC']]

# Add a row for overall cover rate
overall = [['OVERALL STATE COVER RATE', 0.911, 0.890, 0.884, 0.882, 0.880, 0.812, 0.804, 0.802, 0.787, 0.773]]
overall_df = pd.DataFrame(overall, columns=['provider_state', 'MD', 'AK', 'CA', 'MA', 'MT', 'OH', 'VA', 'PA', 'UT', 'DC']).set_index('provider_state')
c_compare_pivot = pd.concat([overall_df, c_compare_pivot])
c_compare_pivot.head()

In [ ]:
# Plot the heatmap
size = (20, 15)
fig, ax = plt.subplots(figsize=size)
sns.heatmap(data=c_compare_pivot, square=True, annot=True, cbar=False, cmap='crest', ax=ax)
plt.tick_params(axis='both', which='major', labelsize=10, labelbottom = False, bottom=False, top = False, labeltop=True)

Different states have different cover rates for top clinical conditions, which explains the by-state difference in overall cover rates. For example, the cover rate for infectious & parasitic diseases is .91 in Maryland, which had the highest overall cover rate, while the cover rate for infectious & parasitic diseases in D.C. is only .77.

#### 3.5 The relationship between Life expectancy/Income and Avg_Total_Payment

In [ ]:
size = (25,8)
fig, ax = plt.subplots(1,2,figsize=size)

total_payment_by_state = inpatient_2015.copy()
total_payment_by_state_clean = total_payment_by_state.groupby('provider_state')['average_total_payments'].mean().to_frame().reset_index()
life_income_payment = life_expectancy.merge(total_payment_by_state_clean,left_on='State',right_on='provider_state', how='inner').drop('provider_state', axis=1)\
                                    .merge(income_state, left_on='State', right_on='abbreviation',how='inner').drop('abbreviation', axis=1)
a = sns.regplot(x='Life Expectancy',y='average_total_payments',data=life_income_payment,ax=ax[0],ci=None)
a.title.set_text('Life expectancy and Avg total payment')
b = sns.regplot(x='household_median_income',y='average_total_payments',data=life_income_payment,ax=ax[1],ci=None)
b.title.set_text('Income and Avg total payment')

plt.suptitle('The Relationship between Life expectancy and Income and Average total payment')

#### 3.6 Machine Learning to Predict Average total payment

In [ ]:
# preprocess data
inpatient_top100_condition = inpatient_2015[inpatient_2015['drg_definition'].isin(top_100_condition['drg_definition'])]
inpatient_top100_condition_income_life = inpatient_top100_condition.merge(income_state,how='left',left_on='provider_state',right_on='abbreviation').drop('abbreviation',axis=1)\
.merge(life_expectancy,how = 'left',left_on = 'provider_state',right_on='State').drop('State',axis=1)
inpatient_top100_condition_income_life.loc[inpatient_top100_condition_income_life['Life Expectancy'].isnull(),'Life Expectancy'] = 78.5

In [ ]:
y = inpatient_top100_condition_income_life['average_total_payments']
X = inpatient_top100_condition_income_life[['provider_state','provider_city','total_discharges','drg_definition','household_median_income','Life Expectancy']]

X = pd.get_dummies(X,drop_first=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state=42)

##### Using Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)
y_pred = reg_all.predict(X_test)
reg_all.score(X_test, y_test)
from sklearn.metrics import mean_squared_error
print('The score is:',reg_all.score(X_test, y_test),'; The MAE is:',mean_absolute_error(y_test, y_pred))

The linearRegression model is not suitable for our data. let's try other model

##### Using Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1, normalize=True)
ridge.fit(X_train, y_train)
y_model = ridge.predict(X_test) 
from sklearn.metrics import mean_absolute_error
print('The score is:',ridge.score(X_test,y_test),'; The MAE is:',mean_absolute_error(y_test, y_model))

In [ ]:
test= X_test.join(y_test).reset_index()
a = test.join(pd.Series(y_model, name='predicted'))
ax1 = sns.distplot(a['average_total_payments'], hist=False, color="r", label="Actual Value")
sns.distplot(a['predicted'], hist=False, color="b", label="Fitted Values" , ax=ax1)
plt.xlim(0,100000)

In [ ]:
plt.scatter(x= a['average_total_payments'], y=a['predicted'])
plt.xlabel('average_total_payments')
plt.ylabel('predicted')
plt.xlim(0,100000)
plt.ylim(0,100000)
plt.show()

##### Using Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
test_size = 0.3, random_state=42)
lasso = Lasso(alpha=0.1, normalize=False)
lasso.fit(Xtrain, ytrain)
lasso_pred = lasso.predict(Xtest)
print('The score is:',lasso.score(Xtest,ytest),'; The MAE is:',mean_absolute_error(ytest, lasso_pred))

In [ ]:
test= Xtest.join(ytest).reset_index()
b = test.join(pd.Series(y_model, name='predicted'))
ax1 = sns.distplot(b['average_total_payments'], hist=False, color="r", label="Actual Value")
sns.distplot(b['predicted'], hist=False, color="b", label="Fitted Values" , ax=ax1)
plt.xlim(0,100000)

In [ ]:
plt.scatter(x=b['average_total_payments'], y=b['predicted'])
plt.xlabel('average_total_payments')
plt.ylabel('predicted')
plt.xlim(0,100000)
plt.ylim(0,100000)
plt.show()

## 4. Conclusions

1. Medicare payment has consistency with the total payment, but cover rate has little to do with medical payments or total payments.
2. Average total payment increased through 2011-2017.
3. Medicare payment dominates the total payment and performs differently in states.After comparing the result to the nationwide medicare payments, about one third of states have a higher medicare payment than the average.
4. The clinical condition is the major factor on cover rate.
5. Base on the regression charts between Life expectancy and Average total payment, we can conclude that there is a positive relationship between Life expectancy and Average total payment. 
           For example, if we have higher life expectancy, the Average total payment will be higher.
6. Base on the regression charts between Income_median and Average total payment, we can conclude that there is a positive relationship between Income_median and Average total payment. 
           For example, if we have higher Income, the Average total payment will be higher.

## 5. References


- https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data
- https://data.census.gov/cedsci/table?q=S19&d=ACS%201-Year%20Estimates%20Subject%20Tables&tid=ACSST1Y2019.S1901
- https://www.cdc.gov/nchs/data-visualization/life-expectancy/ 